In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from pythainlp.corpus import thai_stopwords, thai_words
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from transformers import AutoTokenizer, AutoModel
from pythainlp.tokenize import Tokenizer
from pythainlp.util import dict_trie

import numpy as np
import joblib
import torch 
import re


In [ ]:
def Load_data(filename): 
    import codecs

    with codecs.open(filename, 'r', 'utf-8') as file: 
        return [line.strip() for line in file.readlines()]

In [34]:
def Lemmatization(data_list): 
    word_lemmatization = {
        r'รมว\.กต\.?': 'รัฐมนตรีว่าการกระทรวงการต่างประเทศของไทย',
        r'รมว\.กต': 'รัฐมนตรีว่าการกระทรวงการต่างประเทศของไทย',

        r'กก\.บห\.?': 'คณะกรรมการบริหาร',
        r'กกบห\.': 'คณะกรรมการบริหาร',

        r'ผบ\.สส\.?': 'ผู้บัญชาการทหารสูงสุด',
        r'ผบสส\.': 'ผู้บัญชาการทหารสูงสุด',

        r'ส\.ป\.ส\.ช\.?': 'สำนักงานหลักประกันสุขภาพแห่งชาติ',
        r'สปสช\.': 'สำนักงานหลักประกันสุขภาพแห่งชาติ',

        r'ก\.ส\.ท\.ช\.?': 'คณะกรรมการกิจการกระจายเสียงกิจการโทรทัศน์และกิจการโทรคมนาคมแห่งชาติ',
        r'กสทช\.': 'คณะกรรมการกิจการกระจายเสียงกิจการโทรทัศน์และกิจการโทรคมนาคมแห่งชาติ',

        r'ร\.ม\.ว\.?': 'รัฐมนตรีว่าการ',
        r'รมว\.': 'รัฐมนตรีว่าการ',

        r'ส\.ค\.ร\.?': 'สำนักงานคณะกรรมการนโยบายรัฐวิสาหกิจ',
        r'สคร\.': 'สำนักงานคณะกรรมการนโยบายรัฐวิสาหกิจ',

        r'ค\.ร\.ม\.?': 'คณะรัฐมนตรี',
        r'ครม\.': 'คณะรัฐมนตรี',

        r'ส\.ม\.ช\.?': 'สำนักงานสภาความมั่นคงแห่งชาติ',
        r'สมช\.': 'สำนักงานสภาความมั่นคงแห่งชาติ',

        r'น\.ร\.ต\.?': 'นักเรียนนายร้อยตำรวจ',
        r'นรต\.': 'นักเรียนนายร้อยตำรวจ',

        r'ส\.ส\.ส\.?': 'สำนักงานกองทุนสนับสนุนการสร้างเสริมสุขภาพ',
        r'สสส\.': 'สำนักงานกองทุนสนับสนุนการสร้างเสริมสุขภาพ',

        r'ผ\.บ\.ช\.': 'ผู้บัญชาการ',
        r'ผบช\.': 'ผู้บัญชาการ',

        r'ก\.อ\.ช\.?': 'กองทุนการออมแห่งชาติ',
        r'กอช\.': 'กองทุนการออมแห่งชาติ',

        r'ส\.ป\.ป\.?': 'สาธารณรัฐประชาธิปไตยประชาชน',
        r'สปป\.': 'สาธารณรัฐประชาธิปไตยประชาชน',

        r'ก\.ส\.ร\.?': 'กรมสวัสดิการและคุ้มครองแรงงาน',
        r'กสร\.': 'กรมสวัสดิการและคุ้มครองแรงงาน',

        r'ก\.ท\.ม\.?': 'กรุงเทพมหานคร',
        r'กทม\.': 'กรุงเทพมหานคร',

        r'ก\.ก\.ต\.?': 'คณะกรรมการการเลือกตั้ง',
        r'กกต\.': 'คณะกรรมการการเลือกตั้ง',

        r'อ\.บ\.จ\.?': 'องค์การบริหารส่วนจังหวัด',
        r'อบจ\.': 'องค์การบริหารส่วนจังหวัด',

        r'พ\.ร\.บ\.?': 'พระราชบัญญัติ',
        r'พรบ\.': 'พระราชบัญญัติ',

        r'ผ\.ก\.ก\.?': 'ผู้กำกับการ',
        r'ผกก\.': 'ผู้กำกับการ',

        r'ก\.ก\.ร\.?': 'คณะกรรมการร่วมภาคเอกชน3สถาบัน',
        r'กกร\.': 'คณะกรรมการร่วมภาคเอกชน3สถาบัน',

        r'ส\.พ\.ฐ\.?': 'สำนักงานคณะกรรมการการศึกษาขั้นพื้นฐาน',
        r'สพฐ\.': 'สำนักงานคณะกรรมการการศึกษาขั้นพื้นฐาน',

        r'ม\.ท\.ร\.?': 'มหาวิทยาลัยเทคโนโลยีราชมงคล',
        r'มทร\.': 'มหาวิทยาลัยเทคโนโลยีราชมงคล',

        r'จ\.น\.ท\.?': 'เจ้าหน้าที่',
        r'จนท\.': 'เจ้าหน้าที่',

        r'ป\.ป\.ง\.?': 'สำนักงานป้องกันและปราบปรามการฟอกเงิน',
        r'ปปง\.': 'สำนักงานป้องกันและปราบปรามการฟอกเงิน',

        r'ธ\.ก\.ส\.?': 'ธนาคารเพื่อการเกษตรและสหกรณ์การเกษตร',
        r'ธกส\.': 'ธนาคารเพื่อการเกษตรและสหกรณ์การเกษตร',

        r'ส\.ป\.ส\.?': 'สำนักงานประกันสังคม',
        r'สปส\.': 'สำนักงานประกันสังคม',

        r'พ\.น\.ง\.?': 'พนักงาน',
        r'พนง\.': 'พนักงาน',

        r'ก\.ส\.ท\.?': 'การสื่อสารแห่งประเทศไทย',
        r'กสท\.': 'การสื่อสารแห่งประเทศไทย',

        r'ก\.ฟ\.ภ\.?': 'การไฟฟ้าส่วนภูมิภาค',
        r'กฟภ\.': 'การไฟฟ้าส่วนภูมิภาค',

        r'ร\.ฟ\.ม\.?': 'การรถไฟฟ้าขนส่งมวลชนแห่งประเทศไทย',
        r'รฟม\.': 'การรถไฟฟ้าขนส่งมวลชนแห่งประเทศไทย',

        r'ป\.ช\.ช\.?': 'ประชาชน',
        r'ปชช\.': 'ประชาชน',

        r'ต\.ก\.?': 'กองแผนงานกิจการพิเศษ',
        r'ตก\.': 'กองแผนงานกิจการพิเศษ',

        r'พ\.ม\.?': 'กระทรวงการพัฒนาสังคมและความมั่นคงของมนุษย์',
        r'พม\.': 'กระทรวงการพัฒนาสังคมและความมั่นคงของมนุษย์',
        
        r'ก\.ม\.?': 'กฎหมาย',
        r'กม\.': 'กฎหมาย',

        r'ม\.ท\.?': 'กระทรวงมหาดไทย',
        r'มท\.': 'กระทรวงมหาดไทย',

        r'ต\.ม\.?': 'กองตรวจคนเข้าเมือง',
        r'ตม\.': 'กองตรวจคนเข้าเมือง',

        r'ส\.ธ\.?': 'กระทรวงสาธารณสุข',
        r'สธ\.': 'กระทรวงสาธารณสุข',

        r'อ\.ย\.?': 'สํานักงานคณะกรรมการอาหารและยา',
        r'อย\.': 'สํานักงานคณะกรรมการอาหารและยา',

        r'ผ\.อ\.?': 'ผู้อำนวยการ',
        r'ผอ\.': 'ผู้อำนวยการ',

        r'ผ\.ก\.?': 'กองแผนงานกิจการพิเศษ',
        r'ผก\.': 'กองแผนงานกิจการพิเศษ',

        r'ร\.พ\.?': 'โรงพยาบาล',
        r'รพ\.': 'โรงพยาบาล',

        r'ส\.ภ\.?': 'สถานีตำรวจภูธร',
        r'สภ\.': 'สถานีตำรวจภูธร',

        r'น\.ร\.?': 'นักเรียน',
        r'นร\.': 'นักเรียน',
        
        r'ร\.ร\.?': 'โรงเรียน',
        r'รร\.': 'โรงเรียน',

        r'ท\.ส\.?': 'กระทรวงทรัพยากรธรรมชาติและสิ่งแวดล้อม',
        r'ทส\.': 'กระทรวงทรัพยากรธรรมชาติและสิ่งแวดล้อม',

        r'น\.ช\.?': 'นักโทษชาย',
        r'นช\.': 'นักโทษชาย',

        r'ศ\.ธ\.?': 'กระทรวงศึกษาธิการ',
        r'ศธ\.': 'กระทรวงศึกษาธิการ',

        r'ผ\.บ\.?': 'ผู้บัญชาการ',
        r'ผบ\.': 'ผู้บัญชาการ',

        r'ล\.บ\.?': 'ล้านบาท',
        r'ลบ\.': 'ล้านบาท',

        r'ม\.ค\.?': 'มกราคม', 
        r'มค\.': 'มกราคม',

        r'ก\.พ\.?': 'กุมภาพันธ์',
        r'กพ\.': 'กุมภาพันธ์',

        r'มี\.ค\.?': 'มีนาคม',
        r'มีค\.': 'มีนาคม',

        r'เม\.ย\.?': 'เมษายน',
        r'เมย\.': 'เมษายน',

        r'พ\.ค\.?': 'พฤษภาคม',
        r'พค\.': 'พฤษภาคม',

        r'มิ\.ย\.?': 'มิถุนายน',
        r'มิย\.': 'มิถุนายน',

        r'ก\.ค\.?': 'กรกฎาคม',
        r'กค\.': 'กรกฎาคม',

        r'ส\.ค\.?': 'สิงหาคม',
        r'สค\.': 'สิงหาคม',

        r'ก\.ย\.?': 'กันยายน',
        r'กย\.': 'กันยายน',

        r'ต\.ค\.?': 'ตุลาคม',
        r'ตค\.': 'ตุลาคม',

        r'พ\.ย\.?': 'พฤศจิกายน',
        r'พย\.': 'พฤศจิกายน',

        r'ธ\.ค\.?': 'ธันวาคม',
        r'ธค\.': 'ธันวาคม',

        r'ม\.33': 'มาตรา33',
        r'ม\.39': 'มาตรา39',
        r'ม\.40': 'มาตรา40',
        r'ม\.112': 'มาตรา112',
    }

    for pattern, full_month in word_lemmatization.items():
        data_list = [re.sub(pattern, full_month, t) for t in data_list]

    return data_list

In [35]:
def Remove_Number(data_list): 
    text_no_numbers = [re.sub(r'\d+', '', t) for t in data_list]
    return text_no_numbers

In [36]:
def Remove_Special_character(data_list):
    cleaned_data_list = [re.sub(r'[^a-zA-Z0-9ก-ฮะ-์ํา]+', '', t) for t in data_list]
    return cleaned_data_list

In [37]:
def Tokenization(data_list): 
    custom_words_list = set(thai_words())
    custom_words_list.add('รัฐมนตรีว่าการ')
    custom_words_list.add('สำนักงานกองทุนสนับสนุนการสร้างเสริมสุขภาพ')
    custom_words_list.add('ตำรวจไซเบอร์')
    custom_words_list.add('ไม้มิสวาก')
    custom_words_list.add('ไม่มี')
    custom_words_list.add('คอล')
    custom_words_list.add('ผู้บัญชาการไซเบอร์')
    custom_words_list.add('ของฟรีไม่มีในโลก')
    custom_words_list.add('ล้านบาท')
    custom_words_list.add('ทุบสถิติ')
    custom_words_list.add('ไม่ให้')
    custom_words_list.add('แสนราย')
    custom_words_list.add('พันล้าน')
    custom_words_list.add('กาซ่า')
    custom_words_list.add('กองทุนการออมแห่งชาติ')
    custom_words_list.add('ต้องการ')
    custom_words_list.add('แรงงาน')
    custom_words_list.add('จูราสสิคเวิลด์')
    custom_words_list.add('ไม่ใช่')
    custom_words_list.add('จู่โจม')
    custom_words_list.add('ไม่ให้')
    custom_words_list.add('กรมสวัสดิการและคุ้มครองแรงงาน')
    custom_words_list.add('กรมควบคุมโรค')
    custom_words_list.add('อนามัย')
    custom_words_list.add('บุหรี่ไฟฟ้า')
    custom_words_list.add('องค์กรพลังงาน')
    custom_words_list.add('แล้ว')
    custom_words_list.add('คริป')
    custom_words_list.add('คริปโต')
    custom_words_list.add('กระทรวงการพัฒนาสังคมและความมั่นคงของมนุษย์')
    custom_words_list.add('อันตราย')
    custom_words_list.add('องค์การค้า')
    custom_words_list.add('คริสไรท์')
    custom_words_list.add('ไม่น่า')
    custom_words_list.add('รูบิโอ')
    custom_words_list.add('เนทันยาฮู')
    custom_words_list.add('สีจิ้นผิง')
    custom_words_list.add('ทรีนิตี้')
    custom_words_list.add('เฟนทานิล')
    custom_words_list.add('อ่าว')
    custom_words_list.add('กวนตานาโม')
    custom_words_list.add('กล้าธรรม')
    custom_words_list.add('เซเลนสกี')
    custom_words_list.add('ดีอี')
    custom_words_list.add('ออนแทรีโอ')
    custom_words_list.add('สมศักดิ์')
    custom_words_list.add('อิชิบะ')
    custom_words_list.add('บิ๊กอ้วน')
    custom_words_list.add('บูลลี่')
    custom_words_list.add('ผู้บัญชาการทหารสูงสุด')
    custom_words_list.add('ขึ้น')
    custom_words_list.add('ราชกรมทัณฑ์')
    custom_words_list.add('รายงาน')
    custom_words_list.add('สํานักงานคณะกรรมการอาหารและยา')
    custom_words_list.add('คณะกรรมการกิจการกระจายเสียงกิจการโทรทัศน์และกิจการโทรคมนาคมแห่งชาติ')
    
    trie = dict_trie(dict_source=custom_words_list)
    _tokenizer = Tokenizer(custom_dict=trie, engine='newmm')

    list_data = []
    for i in range(len(data_list)): 
        word = _tokenizer.word_tokenize((data_list[i]))
        word = " ".join(word)
        list_data.append(word)
    return list_data

In [38]:
def Stopwords(data_list): 
    cleaned_data_list = []
    for data in data_list:
        words = " ".join(word for word in data.split() if word not in thai_stopwords())
        cleaned_data_list.append(words)
    return cleaned_data_list

In [39]:
def Lowercase(data_list):
    return [text.lower() for text in data_list]

In [ ]:
def Import_Data(list_fake_news, list_true_news):
    list_fake_news = list_fake_news
    list_true_news = list_true_news

    sentences = list_fake_news + list_true_news
    labels = [0] * len(list_fake_news) + [1] * len(list_true_news)

    x_train, x_test, y_train, y_test = train_test_split(sentences, labels, test_size = 0.1)

    return x_train, x_test, y_train, y_test

In [40]:
def preprocess_text(text):
    text = Lemmatization(text)
    text = Remove_Special_character(text)
    text = Tokenization(text)
    text = Lowercase(text)
    return text

In [41]:
def encode_sentences(sentences, batch_size=16):
    model_name = "airesearch/wangchanberta-base-att-spm-uncased"
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name).to(device)

    count = 0
    all_embeddings = []
    for i in range(0, len(sentences), batch_size):
        batch = sentences[i:i + batch_size]
        batch = preprocess_text(batch)
        # print(batch)
        inputs = tokenizer(batch, padding=True, truncation=True, return_tensors='pt', max_length=512)
        inputs = {key: value.to(device) for key, value in inputs.items()}
        
        with torch.no_grad():
            outputs = model(**inputs)

        batch_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
        all_embeddings.append(batch_embeddings)
        count += 1
        # print(count)

    return np.vstack(all_embeddings)

In [ ]:
def Text_To_BERT(x_train, x_test): 
    x_train_vectorizer = encode_sentences(x_train)
    print('Test x_train_vectorizer Success ✅')
    x_test_vectorizer = encode_sentences(x_test)
    print('Test x_test_vectorizer Success ✅')

    return x_train_vectorizer, x_test_vectorizer

In [ ]:
def Save_Vectorizer(x_train_vectorizer, x_test_vectorizer, y_train, y_test):
    joblib.dump(x_train_vectorizer, 'x_train_wangchan.pkl')
    joblib.dump(x_test_vectorizer, 'x_test_wangchan.pkl')
    joblib.dump(y_train, 'y_train_wangchan.pkl')
    joblib.dump(y_test, 'y_test_wangchan.pkl')


In [ ]:
def Load_Vectorizer(x_train, x_test, y_train, y_test): 
    x_train_vectorizer = joblib.load(x_train)
    x_test_vectorizer = joblib.load(x_test)
    y_train = joblib.load(y_train)
    y_test = joblib.load(y_test)

    return x_train_vectorizer, x_test_vectorizer, y_train, y_test

In [ ]:
list_fake_news = Load_data('fake_news.txt')
list_true_news = Load_data('true_news.txt')


x_train, x_test, y_train, y_test = Import_Data(list_fake_news, list_true_news)
x_train_vectorizer, x_test_vectorizer = Text_To_BERT(x_train, x_test)
Save_Vectorizer(x_train_vectorizer, x_test_vectorizer, y_train, y_test)

In [ ]:
def Logistic_Regression(x_train, x_test, y_train, y_test):
    name = 'Logistic_Regression'
    model = LogisticRegression(max_iter=2000, C=1, tol=0.001)
    model.fit(x_train, y_train) 
    y_pred = model.predict(x_test)  

    return name, y_pred, model

In [ ]:
def Classification_Report(name, y_test, y_pred): 
    accuracy = accuracy_score(y_test, y_pred)
    print(f'ความแม่นยำของโมเดล {name} : {accuracy * 100:.2f}')

    classification = classification_report(y_test, y_pred, target_names=['Fake News', 'True News'])
    print(classification)

    cm = confusion_matrix(y_test, y_pred)
    print("Confusion Matrix:")
    print(cm)

    if cm.shape == (2, 2):
        tn, fp, fn, tp = cm.ravel()
        print(f'True Positive (TP): {tp}')
        print(f'False Positive (FP): {fp}')
        print(f'True Negative (TN): {tn}')
        print(f'False Negative (FN): {fn}')
    else:
        print("Confusion matrix is not binary. Please check the number of classes.")

In [42]:
def Sentence(data_list, model_train):
    global count_true_news, count_fake_news
    sentence_vectorizer = encode_sentences(data_list)
    sentence_pred = model_train.predict(sentence_vectorizer)
    sentence_probs = model_train.predict_proba(sentence_vectorizer)
    
    count_true_news = 0
    count_fake_news = 0

    for i in range(len(sentence_pred)):
        sentence_accuracy = max(sentence_probs[i]) * 100  # ความมั่นใจของโมเดล

        if sentence_pred[i] == 'ข่าวจริง' or sentence_pred[i] == 1:
            # if (len(data_list) <= 1): 
            #     print(f'ข่าวที่ {i + 1}: {data_list[i]} || ข่าวจริง: {sentence_accuracy:.2f}%')
            count_true_news += 1
        else:
            # if (len(data_list) <= 1): 
            #     print(f'ข่าวที่ {i + 1}: {data_list[i]} || ข่าวปลอม:{sentence_accuracy:.2f}%')
            count_fake_news += 1

    labels = ''
    if (sentence_pred[i] == 0): 
        labels = 'ข่าวปลอม'
    else: 
        labels = 'ข่าวจริง'

    return f'ข่าว: {data_list[0]} || {labels}: {sentence_accuracy:.2f}%'

def Test_Fake_News(data_list, model_train): 
    Sentence(data_list, model_train)
    print(f'ในการทดสอบข่าวปลอมทั้งหมด {len(data_list)} ข่าว, โมเดลทำนายผิดว่าเป็นข่าวจริง : {count_true_news} ข่าว')

def Test_True_News(data_list, model_train): 
    Sentence(data_list, model_train)
    print(f'ในการทดสอบข่าวจริงทั้งหมด {len(data_list)} ข่าว, โมเดลทำนายผิดว่าเป็นข่าวปลอม : {count_fake_news} ข่าว')

In [ ]:
x_train, x_test, y_train, y_test = Load_Vectorizer('x_train_wangchan.pkl', 'x_test_wangchan.pkl', 'y_train_wangchan.pkl', 'y_test_wangchan.pkl')
name, y_pred, model_train = Logistic_Regression(x_train, x_test, y_train, y_test)
Classification_Report(name, y_test, y_pred)
joblib.dump(model_train, 'model.lr.pkl')

In [43]:
sentence = str(input('\nEnter Sentence : '))
models = joblib.load('model.lr.pkl')
result = Sentence([sentence], models)
print(result)

c:\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


ข่าว: ‘อิ๊งค์’ ดูความคืบหน้าค้นหาผู้สูญหายตึก สตง.ถล่ม พันธมิตรแรงงานฯ คุกเข่าไหว้ทั้งน้ำตาวอนช่วยเหลือ || ข่าวจริง: 98.52%
